In [1]:

from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import cv2
import numpy as np
import time

In [4]:

# Initialize camera
camera = Camera.instance()
image = widgets.Image(format='jpeg', width=640, height=480)

def process_frame(change):
    Greendetected = False
    
    frame = change['new']

    # Select Region of Interest (ROI) from the frame
    roi = frame[200:250, 0:639]

    # Threshold for black line detection
    Blackline = cv2.inRange(roi, (0, 0, 0), (110, 110, 110))
    Greensign = cv2.inRange(roi, (0, 100, 0), (100, 255, 100))

    # Morphological operations
    kernel = np.ones((7, 7), np.uint8)
    Blackline = cv2.erode(Blackline, kernel, iterations=5)
    Blackline = cv2.dilate(Blackline, kernel, iterations=9)
    Greensign = cv2.erode(Greensign, kernel, iterations=5)
    Greensign = cv2.dilate(Greensign, kernel, iterations=9)

    # Find contours
    contours_blk, hierarchy_blk = cv2.findContours(Blackline.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    countours_grn,hierarchy_grn = cv2.findContours(Greensign.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    if len(countours_grn) > 0:
        Greendetected=True
        x_grn, y_grn, w_grn, h_grn = cv2.boundingRect(countours_grn[0])
        centerx_grn=x_grn+(w_grn//2)
        cv2.line(frame, (centerx_grn, 200), (centerx_grn, 250), (0, 0, 255), 3)

    if len(contours_blk) > 0:
        x_blk, y_blk, w_blk, h_blk = cv2.boundingRect(contours_blk[0])
        centerx_blk=x_blk+(w_blk//2)
        cv2.line(frame, (centerx_blk, 200), (centerx_blk, 250), (255, 0, 0), 3)

        if Greendetected:
            if centerx_grn>centerx_blk:
                cv2.putText(frame,"Turn right", (350,50) ,cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
            else:
                cv2.putText(frame,"Turn left", (50,50) ,cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)

        else :
            setpoint=320
            deviation=centerx_blk-setpoint
            centertext="Deviation = "+str(deviation)
            cv2.putText(frame, centertext,(50,100), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)

    # Update display
    image.value = bgr8_to_jpeg(frame)

display(image)
camera.observe(process_frame, names='value')

Image(value=b'', format='jpeg', height='480', width='640')

In [3]:
camera.stop()